In [1]:
from utils import *

def view(show=False, limit=None):
    pokemons = PokemonKind.objects.all()
    if limit:
        pokemons = pokemons[:limit]
    render('pokemon_list.html', context={'pokemons': pokemons}, show=show)
    
view(show=True, limit=3)

NameError: name 'PokemonKind' is not defined

In [2]:
with count_queries() as m:
    view()

print(m.queries)

Number of queries: 1
['SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind"']


In [3]:
timeit(view)

Ran in: 0.035661999998410465


In [4]:
def trainer_pokemon_view(show=False, limit=5000):
    trainers = Trainer.objects.all()
    if limit:
        trainers = trainers.order_by("?")[:limit]
    render('trainer_pokemons.html', context={'trainers': trainers}, show=show)

trainer_pokemon_view(show=True, limit=3)

In [5]:
with count_queries() as m:
    trainer_pokemon_view(limit=10)

Number of queries: 11


In [6]:
for q in m.queries:
    print(q, end='\n\n')

SELECT "Trainer"."id", "Trainer"."name" FROM "Trainer" ORDER BY RANDOM() ASC  LIMIT 10

SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind" INNER JOIN "Pokemon" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") WHERE "Pokemon"."trainer_id" = 9496

SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind" INNER JOIN "Pokemon" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") WHERE "Pokemon"."trainer_id" = 5121

SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind" INNER JOIN "Pokemon" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") WHERE "Pokemon"."trainer_id" = 7988

SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind" INNER JOIN "Pokemon" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") WHERE "Pokemon"."trainer_id" = 5326

SELECT "Poke

In [5]:
display_template('trainer_pokemons.html')

In [8]:
# timeit(trainer_pokemon_view)

In [9]:
%load_ext sql

In [10]:
%sql sqlite:///pokemon.sqlite

'Connected: @pokemon.sqlite'

In [11]:
query = m.queries[1]
print(query, end='\n\n')
%sql $query

SELECT "PokemonKind"."id", "PokemonKind"."name", "PokemonKind"."type1", "PokemonKind"."type2" FROM "PokemonKind" INNER JOIN "Pokemon" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") WHERE "Pokemon"."trainer_id" = 9496

 * sqlite:///pokemon.sqlite
Done.


id,name,type1,type2
30,Bellsprout,Grass,Poison


In [12]:
%sql SELECT * FROM "Pokemon" INNER JOIN "PokemonKind" ON ("PokemonKind"."id" = "Pokemon"."pokemon_kind_id") LIMIT 10

 * sqlite:///pokemon.sqlite
Done.


trainer_id,place,pokelevel,hp,maxhp,attack,defense,spatk,spdef,speed,pokemon_kind_id,id,id_1,name,type1,type2
0,0,60,157,157,50,68,50,80,116,1,1,1,Smeargle,Normal,None
1,0,30,83,83,27,36,27,42,60,1,2,1,Smeargle,Normal,None
2,0,57,281,281,128,76,128,76,93,2,3,2,Wailord,Water,None
2,1,57,173,173,116,122,150,128,82,3,4,3,Vileplume,Grass,Poison
3,0,47,144,144,139,115,87,97,64,4,5,4,Armaldo,Rock,Bug
3,1,47,154,154,98,113,98,122,62,5,6,5,Cradily,Rock,Grass
4,0,56,170,170,165,137,103,114,75,4,7,4,Armaldo,Rock,Bug
4,1,56,182,182,115,133,115,145,73,5,8,5,Cradily,Rock,Grass
5,0,21,57,57,50,20,39,20,39,6,9,6,Carvanha,Water,Dark
5,1,21,63,63,37,29,39,31,27,7,10,7,Numel,Fire,Ground


In [17]:
with open('utils.py', 'r') as f:
    get_ipython().set_next_input(f.read())

In [ ]:
from IPython.core.display import display as idisplay, HTML
import pprint
from django.template.loader import get_template
from django.db import connection
import timeit as pytimeit


def pretty(*args, **kwargs):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(*args, **kwargs)


def display(html):
    idisplay(HTML(html))


def show_template(template_name):
    template = get_template(template_name)
    idisplay(template.template.source)


def render(template_name, context, show=False):
    template = get_template(template_name)

    html = template.render(context=context)

    if show:
        display(html)


def timeit(func):
    total = pytimeit.timeit(func, number=1)
    print('Ran in:', total)


class count_queries:
    def __init__(self):
        self.queries = []

    def __enter__(self):
        self.initial = len(connection.queries)
        return self

    def __exit__(self, *args, **kwargs):
        self.queries = [q['sql'] for q in connection.queries[self.initial:]]
        print('Number of queries:', len(self.queries))
